# Quais deputados se autobeneficiaram ao votar a favor da lei de improbidade administrativa
O código vai buscar informações, via API da Câmara dos Deputados, da votação do PL n° 2505, de 2021 - sob ID DE VOTAÇÃO: 2184458-184. <br>
Na 2ª etapa, iremos automatizar searchs no e-SAJ do TJ de todos os estados brasileiros e raspar as informações para análise posterior.
## Preparando o ambiente
Para a raspagem dos dados, utilizaremos as bibliotecas Selenium. Para a análise, utilizaremos o Pandas. 
Já para a representação dos dados utilizaremos o matplotlib e Seaborn. 

In [1]:

#Biblioteca para raspagem 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

#Bibliotecas para várias funções relacionadas à tempo
import time
from datetime import datetime

#Bibliotecas para análise dos dados
import pandas as pd 
import numpy as np 
import requests as requests
import matplotlib.pyplot as plt 
import seaborn as sns


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Conexão API Câmara dos Deputados

In [3]:
#Vamos criar nosso cabeçalho para aceitar as requisiões em json
headers = {
    "Accept": "Application/json"
}

### Identificar os id's da PL

In [4]:
votos_pl = requests.get('https://dadosabertos.camara.leg.br/api/v2/proposicoes/2184458/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro', headers=headers)
votos_pl = votos_pl.json()

votos_pl = pd.json_normalize(votos_pl['dados'])
votos_pl.head(6)


,id,uri,data,dataHoraRegistro,siglaOrgao,uriOrgao,uriEvento,proposicaoObjeto,uriProposicaoObjeto,descricao,aprovacao
0,2184458-186,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-186,2021-10-06,2021-10-06T16:58:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63505,None,None,"Aprovada a Redação Final assinada pelo Relator, Dep. Carlos Zarattini (PT-SP).",1.0
1,2184458-184,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184,2021-10-06,2021-10-06T16:57:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63505,None,None,Aprovada a Emenda do Senado Federal nº 8. Sim: 287; não: 133; abstenção: 2; total: 422.,1.0
2,2184458-172,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-172,2021-10-05,2021-10-05T20:05:40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,"Rejeitada a Emenda do Senado Federal nº 4 ao Projeto de Lei nº 2.505, de 2021, ressalvados os destaques. Sim: 162; não: 253; abstenção: 4; total: 419.",0.0
3,2184458-169,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-169,2021-10-05,2021-10-05T19:47:45,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,"Aprovadas as Emendas do Senado Federal nºs 1, 2, 3, 5, 6, 7 e 8 ao Projeto de Lei nº 2.505, de 2021, ressalvados os destaques. Sim: 395; não: 22; abstenção: 3; total: 420.",1.0
4,2184458-161,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-161,2021-10-05,2021-10-05T18:34:35,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,Rejeitado o Requerimento. Sim: 60; não: 346; total: 406.,0.0
5,2301348-3,https://dadosabertos.camara.leg.br/api/v2/votacoes/2301348-3,2021-10-05,2021-10-05T13:23:52,CPD,https://dadosabertos.camara.leg.br/api/v2/orgaos/537480,https://dadosabertos.camara.leg.br/api/v2/eventos/63437,REQ 76/2021 CPD,https://dadosabertos.camara.leg.br/api/v2/proposicoes/2301348,Aprovado.,1.0


### Lista votos

In [5]:
lista_votos = requests.get('https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184/votos', headers=headers)
lista_votos = lista_votos.json()

lista_votos = pd.json_normalize(lista_votos['dados'])
lista_votos

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email
0,Sim,2021-10-06T16:55:06,74352,https://dadosabertos.camara.leg.br/api/v2/deputados/74352,Nilson Pinto,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,PA,56,https://www.camara.leg.br/internet/deputado/bandep/74352.jpg,None
1,Sim,2021-10-06T16:54:47,74366,https://dadosabertos.camara.leg.br/api/v2/deputados/74366,João Campos,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,GO,56,https://www.camara.leg.br/internet/deputado/bandep/74366.jpg,None
2,Sim,2021-10-06T16:54:10,204553,https://dadosabertos.camara.leg.br/api/v2/deputados/204553,Pastor Sargento Isidório,AVANTE,https://dadosabertos.camara.leg.br/api/v2/partidos/36898,BA,56,https://www.camara.leg.br/internet/deputado/bandep/204553.jpg,dep.pastorsargentoisidorio@camara.leg.br
3,Não,2021-10-06T16:54:02,141553,https://dadosabertos.camara.leg.br/api/v2/deputados/141553,Vanderlei Macris,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,SP,56,https://www.camara.leg.br/internet/deputado/bandep/141553.jpg,None
4,Não,2021-10-06T16:53:57,204363,https://dadosabertos.camara.leg.br/api/v2/deputados/204363,Osires Damaso,PSC,https://dadosabertos.camara.leg.br/api/v2/partidos/36833,TO,56,https://www.camara.leg.br/internet/deputado/bandep/204363.jpg,None
...,...,...,...,...,...,...,...,...,...,...,...
418,Não,2021-10-06T16:45:03,204548,https://dadosabertos.camara.leg.br/api/v2/deputados/204548,Heitor Freire,PSL,https://dadosabertos.camara.leg.br/api/v2/partidos/36837,CE,56,https://www.camara.leg.br/internet/deputado/bandep/204548.jpg,None
419,Sim,2021-10-06T16:45:01,76874,https://dadosabertos.camara.leg.br/api/v2/deputados/76874,Marcelo Freixo,PSB,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,RJ,56,https://www.camara.leg.br/internet/deputado/bandep/76874.jpg,None
420,Sim,2021-10-06T16:45:01,160556,https://dadosabertos.camara.leg.br/api/v2/deputados/160556,Padre João,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,MG,56,https://www.camara.leg.br/internet/deputado/bandep/160556.jpg,dep.padrejoao@camara.leg.br
421,Sim,2021-10-06T16:44:59,73801,https://dadosabertos.camara.leg.br/api/v2/deputados/73801,Renildo Calheiros,PCdoB,https://dadosabertos.camara.leg.br/api/v2/partidos/36779,PE,56,https://www.camara.leg.br/internet/deputado/bandep/73801.jpg,dep.renildocalheiros@camara.leg.br


### Informações adicionais dos deputados

In [6]:
nome_deputado = []

for i, row in lista_votos.iterrows():
    deputado_id = row['deputado_.id']
    deputados_infos = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}')
    deputados_infos = deputados_infos.json()
    nome_deputado.append(deputados_infos['dados']['nomeCivil'])
    print(deputado_id)

lista_votos['NomeCivilDeputado'] = nome_deputado

74352
74366
204553
141553
204363
160758
204505
178934
178884
217480
204547
204414
178896
178927
204364
204387
204531
160592
141516
204473
178996
204482
160591
204395
141513
141458
178986
141434
157130
204559
160976
196358
178937
178956
204527
74090
204398
204428
141459
74317
204377
204515
178933
73692
204539
160610
198783
204392
74376
166402
160553
179001
74537
74253
160527
217330
74439
178910
74646
204571
73486
74052
209189
204449
204431
178964
178921
204488
66179
90201
204386
74159
197438
178970
204534
164359
160601
178985
178912
74158
204511
204360
204408
204475
73466
74419
178873
160528
204502
141405
204467
178843
204518
178871
73788
160510
133439
160508
177282
204532
73604
204427
178909
204522
204446
141391
204410
160604
191945
204479
204405
178835
195866
178948
204369
215042
204413
160666
204481
160642
204366
204537
204485
74749
141523
141408
143084
160665
204407
75431
204497
204538
205548
115746
204566
92776
178939
204418
178832
204393
204498
74467
194260
74254
204372
136811
204

In [7]:
lista_votos.query('`deputado_.nome`.str.contains("Passos")')

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email,NomeCivilDeputado
328,Sim,2021-10-06T16:45:41,178981,https://dadosabertos.camara.leg.br/api/v2/deputados/178981,Herculano Passos,MDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36899,SP,56,https://www.camara.leg.br/internet/deputado/bandep/178981.jpg,None,HERCULANO CASTILHO PASSOS JUNIOR


In [8]:
lista_votos.query('tipoVoto == "Sim"')['deputado_.siglaUf'].value_counts()

deputado_.siglaUf
MG    32
SP    29
BA    28
RJ    20
RS    19
PE    17
PR    15
CE    12
MA    11
GO     9
PA     9
PI     8
AC     8
SE     7
TO     6
AP     6
RN     6
AL     6
SC     6
PB     6
RR     5
DF     4
MS     4
MT     4
RO     4
ES     3
AM     3
Name: count, dtype: int64

# Inicialização do raspador

In [9]:
teste_processo = [] 
teste_classe = []
teste_assunto = []

driver = webdriver.Chrome()
driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')
WebDriverWait(driver, 5)

driver.maximize_window() 
driver.implicitly_wait(5) 

botao = driver.find_element(By.ID, "cbPesquisa")
select = Select(botao)
select.select_by_index(1)


driver.find_element(By.ID, "campo_NMPARTE").send_keys("HERCULANO CASTILHO PASSOS JUNIOR")
driver.find_element(By.ID, "pesquisarPorNomeCompleto").click()
driver.find_element(By.ID, "botaoConsultarProcessos").click()
driver.find_element(By.ID, "contadorDeProcessos").text

WebDriverWait(driver, 5)

lista_processo = driver.find_elements(By.CLASS_NAME, "linkProcesso")
classe_processo = driver.find_elements(By.CLASS_NAME, "classeProcesso")
assunto_processo = driver.find_elements(By.CLASS_NAME, "assuntoPrincipalProcesso")

count = 1
for i in lista_processo: 
    print(count,
          i.text)
    count +=1
    teste_processo.append(i.text)

for p in classe_processo:
    teste_classe.append(p.text)

for a in assunto_processo:
    teste_assunto.append(a.text)

try:
    botao = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[2]/ul/li[2]/a')
    botao.click()
    
    count = 1
    for i in lista_processo: 
        print(count,
              i.text)
        count +=1
        teste_processo.append(i.text)
        
        for p in classe_processo:
            teste_classe.append(p.text)
            
        for p in classe_processo:
            teste_classe.append(p.text)
        
        for a in assunto_processo:
            teste_assunto.append(a.text)
    
except NoSuchElementException:
    print("Apenas uma página")

WebDriverWait(driver, 5)

1 1011194-58.2024.8.26.0286
2 1008554-82.2024.8.26.0286
3 1007967-60.2024.8.26.0286
4 1000795-67.2024.8.26.0286
5 1506372-03.2023.8.26.0286
6 1506371-18.2023.8.26.0286
7 1506370-33.2023.8.26.0286
8 1507128-80.2021.8.26.0286
9 1007909-62.2021.8.26.0286
10 1008243-33.2020.8.26.0286
11 1006055-67.2020.8.26.0286
12 1000498-02.2020.8.26.0286
13 1010456-17.2017.8.26.0286
14 1503768-79.2017.8.26.0286
15 1003632-76.2016.8.26.0286
16 1003176-29.2016.8.26.0286
17 1000406-63.2016.8.26.0286
18 1002493-60.2014.8.26.0286
19 0012415-16.2012.8.26.0286
20 0006240-06.2012.8.26.0286
21 0010357-74.2011.8.26.0286
22 0005113-67.2011.8.26.0286
23 0004743-88.2011.8.26.0286
24 0014629-53.2007.8.26.0286
25 0013431-78.2007.8.26.0286


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF697DF02F5+28725]
	(No symbol) [0x00007FF697D52AE0]
	(No symbol) [0x00007FF697BE510A]
	(No symbol) [0x00007FF697BF98C4]
	(No symbol) [0x00007FF697BF83D1]
	(No symbol) [0x00007FF697BECA89]
	(No symbol) [0x00007FF697BEA7E2]
	(No symbol) [0x00007FF697BEEABC]
	(No symbol) [0x00007FF697BEEB8F]
	(No symbol) [0x00007FF697C3313F]
	(No symbol) [0x00007FF697C5FFAA]
	(No symbol) [0x00007FF697C2BF86]
	(No symbol) [0x00007FF697C601C0]
	(No symbol) [0x00007FF697C80181]
	(No symbol) [0x00007FF697C5FD53]
	(No symbol) [0x00007FF697C2A0E3]
	(No symbol) [0x00007FF697C2B471]
	GetHandleVerifier [0x00007FF69811F30D+3366989]
	GetHandleVerifier [0x00007FF6981312F0+3440688]
	GetHandleVerifier

# Rodagem Automatizada

In [10]:
bd_deputados_sp = lista_votos.query('`deputado_.siglaUf` == "SP" and tipoVoto == "Sim"')
bd_deputados_sp

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email,NomeCivilDeputado
26,Sim,2021-10-06T16:51:53,178986,https://dadosabertos.camara.leg.br/api/v2/deputados/178986,Nilto Tatto,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,SP,56,https://www.camara.leg.br/internet/deputado/bandep/178986.jpg,dep.niltotatto@camara.leg.br,NILTO IGNACIO TATTO
30,Sim,2021-10-06T16:51:37,160976,https://dadosabertos.camara.leg.br/api/v2/deputados/160976,Tiririca,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/160976.jpg,dep.tiririca@camara.leg.br,FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
34,Sim,2021-10-06T16:51:12,204527,https://dadosabertos.camara.leg.br/api/v2/deputados/204527,Geninho Zuliani,DEM,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204527.jpg,None,EUGENIO JOSÉ ZULIANI
76,Sim,2021-10-06T16:49:06,160601,https://dadosabertos.camara.leg.br/api/v2/deputados/160601,Pr. Marco Feliciano,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/160601.jpg,dep.pr.marcofeliciano@camara.leg.br,MARCO ANTÔNIO FELICIANO
80,Sim,2021-10-06T16:49:00,204511,https://dadosabertos.camara.leg.br/api/v2/deputados/204511,David Soares,DEM,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204511.jpg,dep.davidsoares@camara.leg.br,DAVID BEZERRA RIBEIRO SOARES
100,Sim,2021-10-06T16:48:23,73604,https://dadosabertos.camara.leg.br/api/v2/deputados/73604,Rui Falcão,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,SP,56,https://www.camara.leg.br/internet/deputado/bandep/73604.jpg,dep.ruifalcao@camara.leg.br,RUI GOETHE DA COSTA FALCÃO
121,Sim,2021-10-06T16:47:41,204537,https://dadosabertos.camara.leg.br/api/v2/deputados/204537,Enrico Misasi,PV,https://dadosabertos.camara.leg.br/api/v2/partidos/36851,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204537.jpg,None,ENRICO VAN BLARCUM DE GRAAFF MISASI
122,Sim,2021-10-06T16:47:40,204485,https://dadosabertos.camara.leg.br/api/v2/deputados/204485,Luiz Carlos Motta,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204485.jpg,dep.luizcarlosmotta@camara.leg.br,LUIZ CARLOS MOTTA
147,Sim,2021-10-06T16:47:18,217036,https://dadosabertos.camara.leg.br/api/v2/deputados/217036,Henrique do Paraíso,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,SP,56,https://www.camara.leg.br/internet/deputado/bandep/217036.jpg,None,HENRIQUE STEIN SCIASCIO
163,Sim,2021-10-06T16:47:06,73441,https://dadosabertos.camara.leg.br/api/v2/deputados/73441,Celso Russomanno,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,SP,56,https://www.camara.leg.br/internet/deputado/bandep/73441.jpg,dep.celsorussomanno@camara.leg.br,CELSO UBIRAJARA RUSSOMANNO


In [19]:
driver = webdriver.Chrome()
driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')
driver.maximize_window()

# --- LISTAS PARA ARMAZENAR DADOS ---
nome_deputados_lista = []
partidos_lista = []
num_processos_lista = []
classeprocesso_lista = []
assuntoprocesso_lista = []

def realizar_pesquisa(driver, nome_deputado):
    """Realiza a pesquisa por nome completo no site."""
    try:
        botao_tipo_pesquisa = driver.find_element(By.ID, "cbPesquisa")
        select_tipo_pesquisa = Select(botao_tipo_pesquisa)
        select_tipo_pesquisa.select_by_index(1)  # Seleciona para pesquisar por NOME

        campo_nome = driver.find_element(By.ID, "campo_NMPARTE")
        campo_nome.clear()
        campo_nome.send_keys(nome_deputado)
        
        driver.implicitly_wait(15)
        
        driver.find_element(By.ID, "pesquisarPorNomeCompleto").click() #Seleciona a caixa para pesquisar por nome completo
        
        driver.implicitly_wait(10)

        driver.find_element(By.ID, "botaoConsultarProcessos").click() #Clica no botão para pesquisar
        
    except NoSuchElementException:
        print(f"Erro ao realizar pesquisa para {nome_deputado}")
        return False  # Indica falha na pesquisa
    return True # Indica sucesso na pesquisa


def extrair_dados_da_tabela(driver, nome_deputado, partido_deputado):
    """Extrai dados da tabela de processos, se presente."""
    nome_lista_temp = []
    partidos_lista_temp = []
    num_processos_lista_temp = []
    classeprocesso_lista_temp = []
    assuntoprocesso_lista_temp = []

    try:
        lista_processo_elementos = driver.find_elements(By.CLASS_NAME, "linkProcesso")
        classe_processo_elementos = driver.find_elements(By.CLASS_NAME, "classeProcesso")
        assunto_processo_elementos = driver.find_elements(By.CLASS_NAME, "assuntoPrincipalProcesso")

        if not lista_processo_elementos: # Verifica se encontrou processos
            print(f"Nenhum processo encontrado na tabela para {nome_deputado}")
            return [], [], [], [], []

        for i in range(len(lista_processo_elementos)):
            nome_lista_temp.append(nome_deputado)
            partidos_lista_temp.append(partido_deputado)
            num_processos_lista_temp.append(lista_processo_elementos[i].text)
            classeprocesso_lista_temp.append(classe_processo_elementos[i].text)
            assuntoprocesso_lista_temp.append(assunto_processo_elementos[i].text)

    except NoSuchElementException:
        print(f"Erro ao extrair dados da tabela para {nome_deputado}")
        return [], [], [], [], [] # Retorna listas vazias em caso de erro

    return nome_lista_temp, partidos_lista_temp, num_processos_lista_temp, classeprocesso_lista_temp, assuntoprocesso_lista_temp


def extrair_detalhes_processo(driver, nome_deputado, partido_deputado):
    """Extrai detalhes do processo se estiver na página 'show.do?'."""
    try:
        nprocesso = driver.find_element(By.ID, "numeroProcesso").text
        processo_clase = driver.find_element(By.ID, "labelClasseProcesso").text
        processo_assunto = driver.find_element(By.ID, "assuntoProcesso").text

        nome_deputados_lista.append(nome_deputado)
        partidos_lista.append(partido_deputado)
        num_processos_lista.append(nprocesso)
        classeprocesso_lista.append(processo_clase)
        assuntoprocesso_lista.append(processo_assunto)

    except NoSuchElementException:
        print(f"Erro ao extrair detalhes do processo para {nome_deputado}")


def voltar_para_pesquisa(driver):
    """Limpa o campo de pesquisa para a próxima busca."""
    try:
        campo_nome = driver.find_element(By.ID, "campo_NMPARTE")
        campo_nome.clear()

        time.sleep(10)
        
    except NoSuchElementException:
        print("Erro ao limpar campo de pesquisa.")


# --- LOOP PRINCIPAL ---
for i, row in bd_deputados_sp.iterrows():  
    nome_deputado = row['NomeCivilDeputado']
    partido_deputado = row['deputado_.siglaPartido']

    print(f"Pesquisando por: {nome_deputado} ({partido_deputado})")

    if realizar_pesquisa(driver, nome_deputado): # Realiza pesquisa e verifica sucesso
        url = driver.current_url
        if "show.do?" in str(url):
            extrair_detalhes_processo(driver, nome_deputado, partido_deputado)
            try:
                driver.find_element(By.CLASS_NAME, 'unj-link-back').click() # Voltar para lista
            except NoSuchElementException:
                print("Erro ao clicar em 'Voltar' para lista de processos.")

        else: # Página de lista de processos (ou "nenhum processo encontrado")
            n, pa, np, cp, ap = extrair_dados_da_tabela(driver, nome_deputado, partido_deputado)
            nome_deputados_lista.extend(n)
            partidos_lista.extend(pa)
            num_processos_lista.extend(np)
            classeprocesso_lista.extend(cp)
            assuntoprocesso_lista.extend(ap)

        voltar_para_pesquisa(driver) # Limpa campo para próxima pesquisa
    else:
        print(f"Falha na pesquisa inicial para {nome_deputado}. Pulando para o próximo deputado.")

driver.quit()

Pesquisando por: NILTO IGNACIO TATTO (PT)
Nenhum processo encontrado na tabela para NILTO IGNACIO TATTO
Pesquisando por: FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA (PL)
Nenhum processo encontrado na tabela para FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
Pesquisando por: EUGENIO JOSÉ ZULIANI (DEM)
Pesquisando por: MARCO ANTÔNIO FELICIANO (PL)
Pesquisando por: DAVID BEZERRA RIBEIRO SOARES (DEM)
Pesquisando por: RUI GOETHE DA COSTA FALCÃO (PT)
Nenhum processo encontrado na tabela para RUI GOETHE DA COSTA FALCÃO
Pesquisando por: ENRICO VAN BLARCUM DE GRAAFF MISASI (PV)
Pesquisando por: LUIZ CARLOS MOTTA (PL)
Pesquisando por: HENRIQUE STEIN SCIASCIO (REPUBLICANOS)
Pesquisando por: CELSO UBIRAJARA RUSSOMANNO (REPUBLICANOS)
Pesquisando por: LUIZ PAULO TEIXEIRA FERREIRA (PT)
Pesquisando por: VINICIUS RAPOZO DE CARVALHO (REPUBLICANOS)
Nenhum processo encontrado na tabela para VINICIUS RAPOZO DE CARVALHO
Pesquisando por: PAULO ROBERTO FREIRE DA COSTA (PL)
Pesquisando por: BENEDITO ROBERTO ALVES 

In [20]:
data = {'Nome':nome_deputados_lista, 'Partido':partidos_lista, 'Processo':num_processos_lista, 'Classe':classeprocesso_lista, 'Assunto':assuntoprocesso_lista}

bd_teste = pd.DataFrame(data)

bd_teste

,Nome,Partido,Processo,Classe,Assunto
0,EUGENIO JOSÉ ZULIANI,DEM,1037674-83.2023.8.26.0100,Procedimento Comum Cível,DIREITO PROCESSUAL CIVIL E DO TRABALHO
1,EUGENIO JOSÉ ZULIANI,DEM,1000297-15.2025.8.26.0066,Ação Popular,Atos Administrativos
2,EUGENIO JOSÉ ZULIANI,DEM,1000426-85.2025.8.26.0400,Mandado de Segurança Cível,Concurso Público / Edital
3,EUGENIO JOSÉ ZULIANI,DEM,1001882-07.2024.8.26.0400,Procedimento do Juizado Especial Cível,Indenização por Dano Moral
4,EUGENIO JOSÉ ZULIANI,DEM,1001632-71.2024.8.26.0400,"Crimes de Calúnia, Injúria e Difamação de Competência do Juiz Singular",Simples
...,...,...,...,...,...
187,LUIZA ERUNDINA DE SOUSA,PSOL,0728959-30.1990.8.26.0053,Ação Popular,
188,LUIZA ERUNDINA DE SOUSA,PSOL,0707367-61.1989.8.26.0053,Ação Popular,Atos Administrativos
189,CARLOS ALBERTO ROLIM ZARATTINI,PT,0007357-67.2003.8.26.0053,Ação Popular,Utilização de bens públicos
190,CARLOS ALBERTO ROLIM ZARATTINI,PT,0418437-07.1996.8.26.0053,Cumprimento de sentença,Improbidade Administrativa


In [21]:
bd_teste.query('Nome.str.contains("PASSOS")')

,Nome,Partido,Processo,Classe,Assunto
144,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1011194-58.2024.8.26.0286,Mandado de Segurança Cível,Garantias Constitucionais
145,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1008554-82.2024.8.26.0286,Procedimento Comum Cível,Indenização por Dano Moral
146,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1007967-60.2024.8.26.0286,Procedimento Comum Cível,Contratos Administrativos
147,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1000795-67.2024.8.26.0286,Procedimento Comum Cível,Indenização por Dano Moral
148,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1506372-03.2023.8.26.0286,Execução Fiscal,IPTU/ Imposto Predial e Territorial Urbano
149,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1506371-18.2023.8.26.0286,Execução Fiscal,IPTU/ Imposto Predial e Territorial Urbano
150,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1506370-33.2023.8.26.0286,Execução Fiscal,IPTU/ Imposto Predial e Territorial Urbano
151,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1507128-80.2021.8.26.0286,Execução Fiscal,IPTU/ Imposto Predial e Territorial Urbano
152,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1007909-62.2021.8.26.0286,Ação Civil Pública,Improbidade Administrativa
153,HERCULANO CASTILHO PASSOS JUNIOR,MDB,1008243-33.2020.8.26.0286,Ação Civil Pública,Dano ao Erário


In [22]:
bd_teste['Assunto'].value_counts()

Assunto
Indenização por Dano Moral                                   26
                                                             17
Improbidade Administrativa                                   14
IPTU/ Imposto Predial e Territorial Urbano                   11
Dano ao Erário                                                8
                                                             ..
Sucessões                                                     1
Crimes contra a Fé Pública                                    1
Inexequibilidade do Título / Inexigibilidade da Obrigação     1
Citação                                                       1
Utilização de bens públicos                                   1
Name: count, Length: 72, dtype: int64

In [ ]:
bd_teste['AssuntoProcesso'].value_counts().nlargest(50)

AssuntoProcesso
Indenização por Dano Moral                   27
Gratificações de Atividade                   25
Não Discriminação                            25
DIREITO PROCESSUAL CIVIL E DO TRABALHO       14
                                             12
Violação aos Princípios Administrativos      12
Antecipação de Tutela / Tutela Específica     6
Prestação de Serviços                         5
ISS/ Imposto sobre Serviços                   2
Indenização por Dano Material                 1
Name: count, dtype: int64